In [17]:
#import libraries used
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import re
import requests
from itertools import chain
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import time

#setting the dimension for the data frame
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [18]:
#open up google maps in test environment set up
driver = webdriver.Chrome('C:/Users/nabil/Desktop/intern/web_scaping/chromedriver.exe')

#set the desired location 
driver.get('https://www.google.com.my/maps/place/Penang/@5.353988,100.2229146,11z/data=!3m1!4b1!4m5!3m4!1s0x304a9403095390dd:0x29c305a5f38f633f!8m2!3d5.4163935!4d100.3326786')

In [19]:
#define the target input to extract from 
inputbox = driver.find_element_by_xpath('//input[@id="searchboxinput"]')
inputbox.send_keys("plumber")
enterbutton = driver.find_element_by_xpath('//button[@id="searchbox-searchbutton"]')
enterbutton.click()

In [20]:
#set dataframe and the target number
vendorlist = pd.DataFrame(columns = ['name', 'service', 'contact', 'address'])
target =100
while target > len(vendorlist):
    elements = [e for e in driver.
            find_elements_by_xpath(
                '//div[@class="section-result-content"]')]

    for i in range(np.min([target-len(vendorlist), len(elements)])):
#         print(target-len(vendorlist))
#         print(len(elements))
#         print(i)
        try:
            element = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located(
                    (By.XPATH, 
                     '//div[@class="section-result-content"]') ) )
        except:
            print('issue 1')
            
        try:
            elements = [e for e in driver.
                        find_elements_by_xpath(
                            '//div[@class="section-result-content"]')]
            elements[i].click()
            
        except:
            time.sleep(3)
            elements = [e for e in driver.
                        find_elements_by_xpath(
                            '//div[@class="section-result-content"]')]
            elements[i].click()
        
        try:
            element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//div[@class="section-hero-header-title-description"]/div')) )
        except:
            print('issue 2')

        try:
            name = driver.find_element_by_xpath('//div[@class="section-hero-header-title-description"]/div').text
        except:
            name = None
        try:
            service = driver.find_elements_by_xpath('//button[@jsan="7.widget-pane-link,0.jsaction"]')[0].text
        except:
            service = None
        try:
            if re.sub('[^A-Za-z0-9]+', '', driver.find_elements_by_xpath('//div[@class="ugiz4pqJLAG__primary-text gm2-body-2"]')[0].text).isnumeric():
                contact = re.sub('[^A-Za-z0-9]+', '', driver.find_elements_by_xpath('//div[@class="ugiz4pqJLAG__primary-text gm2-body-2"]')[0].text)
            elif re.sub('[^A-Za-z0-9]+', '', driver.find_elements_by_xpath('//div[@class="ugiz4pqJLAG__primary-text gm2-body-2"]')[1].text).isnumeric():
                contact = re.sub('[^A-Za-z0-9]+', '', driver.find_elements_by_xpath('//div[@class="ugiz4pqJLAG__primary-text gm2-body-2"]')[1].text)
            elif re.sub('[^A-Za-z0-9]+', '', driver.find_elements_by_xpath('//div[@class="ugiz4pqJLAG__primary-text gm2-body-2"]')[2].text).isnumeric():
                contact = re.sub('[^A-Za-z0-9]+', '', driver.find_elements_by_xpath('//div[@class="ugiz4pqJLAG__primary-text gm2-body-2"]')[2].text)
            elif re.sub('[^A-Za-z0-9]+', '', driver.find_elements_by_xpath('//div[@class="ugiz4pqJLAG__primary-text gm2-body-2"]')[3].text).isnumeric():
                contact = re.sub('[^A-Za-z0-9]+', '', driver.find_elements_by_xpath('//div[@class="ugiz4pqJLAG__primary-text gm2-body-2"]')[3].text)
            elif re.sub('[^A-Za-z0-9]+', '', driver.find_elements_by_xpath('//div[@class="ugiz4pqJLAG__primary-text gm2-body-2"]')[4].text).isnumeric():
                contact = re.sub('[^A-Za-z0-9]+', '', driver.find_elements_by_xpath('//div[@class="ugiz4pqJLAG__primary-text gm2-body-2"]')[4].text)
            
        except:
            contact = None
        try:
            adress_check = re.sub('[^A-Za-z0-9]+', '', driver.find_elements_by_xpath('//div[@class="ugiz4pqJLAG__primary-text gm2-body-2"]')[0].text).isalnum()
            if adress_check:
                address = driver.find_elements_by_xpath('//div[@class="ugiz4pqJLAG__primary-text gm2-body-2"]')[0].text
        except:
            address = None

        vendorlist.loc[len(vendorlist)] = [name, service, contact, address]

        back2result = driver.find_element_by_xpath('//button[@class="section-back-to-list-button blue-link noprint"]')
        back2result.click()

    
    try:
        element = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, '//button[@aria-label=" Next page "]'))
        )
        element.click()
        time.sleep(3)
    except:
        print('issue 3')
        break
        


issue 3


In [21]:
# print vendorlist service details
vendorlist

,name,service,contact,address
0,Penang Plumbing Service,Plumber,0195532677,019-553 2677
1,Penang Plumbing Service GK IMPIAN ENTERPRISE,Plumber,0164333660,"28, JALAN CHEAH PHEE CHEOK TANJONG BUNGAH, 112..."
2,Penang Plumbing 1000 Plumbing,Plumber,0164441000,"Island Glades, 16 Lingtang Delima,, Penang, 11..."
3,Ace Plumbers Service,Plumber,0164408838,"Penang, George Town, Jelutong, Taman Hutchings..."
4,Penang Plumbing Service,Plumber,0164393332,"No 30, Lorong tangling indah 3, taman tangling..."
5,Yip Plumbing Service Co,Plumber,042619472,"8B, Lebuh Carnarvon, 10100, Georgetown, Pulau ..."
6,Pinang Plumbing Engineering,Plumber,046588418,"No. 205, Jalan Utara, 11700, Gelugor, Pulau Pi..."
7,Seong Wei Plumbing Service,Plumber,0164440688,"No. 197, Jalan Cy Choy, Georgetown, 10300, Pul..."
8,Choon Plumbing & Sanitary Engineering,Plumber,None,"No. 2, BA, Jalan Gottlieb, 10350, Georgetown, ..."
9,Chan Plumbing,Plumber,042279318,"12A, Jalan Kelantan, 10050 George Town, Pulau ..."


In [22]:
#to check vendorlist service
vendorlist2 = vendorlist
print(vendorlist2.service.unique())

['Plumber' 'Appliance repair service' 'Furniture store'
 'Plumbing supply store' None 'Hardware store' 'Manufacturer'
 'Air conditioning contractor' 'Engineer' 'Cleaning service']


In [24]:
#to remove vendorlist service unique
vendorlist = vendorlist[~vendorlist['service'].isin(['Appliance repair service', 'Furniture store',
 'Plumbing supply store', None, 'Hardware store', 'Manufacturer',
 'Air conditioning contractor', 'Engineer' ,'Cleaning service'])]

In [25]:
#check again the vendorlist service unique
vendorlist2 = vendorlist
print(vendorlist2.service.unique())

['Plumber']


In [26]:
# print(vendorlist.service.unique())
vendorlist

,name,service,contact,address
0,Penang Plumbing Service,Plumber,0195532677,019-553 2677
1,Penang Plumbing Service GK IMPIAN ENTERPRISE,Plumber,0164333660,"28, JALAN CHEAH PHEE CHEOK TANJONG BUNGAH, 112..."
2,Penang Plumbing 1000 Plumbing,Plumber,0164441000,"Island Glades, 16 Lingtang Delima,, Penang, 11..."
3,Ace Plumbers Service,Plumber,0164408838,"Penang, George Town, Jelutong, Taman Hutchings..."
4,Penang Plumbing Service,Plumber,0164393332,"No 30, Lorong tangling indah 3, taman tangling..."
5,Yip Plumbing Service Co,Plumber,042619472,"8B, Lebuh Carnarvon, 10100, Georgetown, Pulau ..."
6,Pinang Plumbing Engineering,Plumber,046588418,"No. 205, Jalan Utara, 11700, Gelugor, Pulau Pi..."
7,Seong Wei Plumbing Service,Plumber,0164440688,"No. 197, Jalan Cy Choy, Georgetown, 10300, Pul..."
8,Choon Plumbing & Sanitary Engineering,Plumber,None,"No. 2, BA, Jalan Gottlieb, 10350, Georgetown, ..."
9,Chan Plumbing,Plumber,042279318,"12A, Jalan Kelantan, 10050 George Town, Pulau ..."


In [30]:
vendorlistplay = vendorlist2
print(vendorlistplay.dtypes)
# print(len(vendorlistplay))
vendorlistplay.drop_duplicates(inplace = True)
# print(len(vendorlistplay))
vendorlistplay.drop_duplicates(subset = ['contact'], inplace = True)
# print(len(vendorlistplay))
vendorlistplay.dropna(subset=['contact'], inplace = True)
# unrelatedtext = ['department', 'uphols', 'hvac', 'plumber', 'vacuum', 'sew', 'audio', 'leather', 'auto', 'paint', 'shoe', 'car', 'tire', 'video', 'camera', 'office', 'furniture', 'handb', 'isuzu', 'reno', 'janitor', 'construct', 'roof', 'contract', 'manufact', 'duct', 'computer', 'fire', 'indust', 'plumb', 'print', 'researc', 'tele', 'tank']
# vendorlistplay['unrelated'] = vendorlistplay.service.apply(lambda x:any(txt in x.lower() for txt in unrelatedtext))
# display(vendorlistplay.groupby(by = ['unrelated','service']).agg(vendor = ('name', 'count')).sort_values(['unrelated','vendor'], ascending = False))
# vendorlistplay = vendorlistplay[vendorlistplay.unrelated == False].reset_index(drop = True)
# print(len(vendorlistplay))
vendorlistplay.reset_index(drop = True, inplace = True)

name       object
service    object
contact    object
address    object
dtype: object


C:\Users\nabil\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\nabil\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\nabil\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [32]:
#export file to excel xlsx
cleanvendorlist = vendorlistplay
# cleanvendorlist = vendorlistplay[vendorlistplay.existing == False]
# cleanvendorlist = cleanvendorlist[cleanvendorlist.previous == False]
# cleanvendorlist = cleanvendorlist[cleanvendorlist.unrelated == False]
cleanvendorlist.contact = cleanvendorlist.contact.apply(lambda x:str(x))
cleanvendorlist.to_excel("Plumber.xlsx", index = False, float_format='%f')
print(len(cleanvendorlist))

43


In [33]:
cleanvendorlist

,name,service,contact,address
0,Penang Plumbing Service,Plumber,0195532677,019-553 2677
1,Penang Plumbing Service GK IMPIAN ENTERPRISE,Plumber,0164333660,"28, JALAN CHEAH PHEE CHEOK TANJONG BUNGAH, 112..."
2,Penang Plumbing 1000 Plumbing,Plumber,0164441000,"Island Glades, 16 Lingtang Delima,, Penang, 11..."
3,Ace Plumbers Service,Plumber,0164408838,"Penang, George Town, Jelutong, Taman Hutchings..."
4,Penang Plumbing Service,Plumber,0164393332,"No 30, Lorong tangling indah 3, taman tangling..."
5,Yip Plumbing Service Co,Plumber,042619472,"8B, Lebuh Carnarvon, 10100, Georgetown, Pulau ..."
6,Pinang Plumbing Engineering,Plumber,046588418,"No. 205, Jalan Utara, 11700, Gelugor, Pulau Pi..."
7,Seong Wei Plumbing Service,Plumber,0164440688,"No. 197, Jalan Cy Choy, Georgetown, 10300, Pul..."
8,Chan Plumbing,Plumber,042279318,"12A, Jalan Kelantan, 10050 George Town, Pulau ..."
9,Ah Chai Plumbing Services,Plumber,0164929695,"11900 Bayan Lepas, Penang"
